# _Manifest File Based_


### 1. What is Manifest File in Android Apps ?
        Manifest file is Like an Identification for a application just like a Id required for humans. It is basically a declaration sheet that tells the
        program important details about the Application, package etc, so it knows how to handle them.

        It contains :
        1. App Meta Data (name, icons, version, min Sdk, etc...)
        2. Permissions (all the permissions are declared here)
        3. Activities (all the activity information must be present in the manifest file)
        4. Services, Stattic BroadCasts and other Configurations...


### 2. Can there be multiple Manifest File in a single App ?
        Yes, there can be multiple manifest file in a single application. Each Module has its own manifest and so does the library dependencies.
        When we build, gradle MERGES all the manifest files according to some MERGE RULES.

# **Activity Based**

## Navigation and Transition Between Activities

### Explain how navigation is handled between 2 activities !

    A. If we DON'T expect the result back -
        - val intent = Intent(this, SignInAct :: class.java)
            startActivity(intent)
        - if we wanna send some data then we can use the .apply{} extention function
            Intent(this, SignInAct :: class.java).apply {
                putExtra(Intent.Extra_Email, recipientArray)
            }

    B. If we expect the result Back -
        - startActivityForResult(Intent,int) // here int is the requestCode
        - result comes through onActivityResult(int,int,Intent) // requestCode, ResultCode, data


 ### Explain how the transition takes place b/w 2 Activities ?

    - A Lanunches B
    - A's onPause() is called
    - B's onCreate(), onStart(), onResume() is executed
    - if A is not at all visible, onStop() executed


### During the Config Change, what is the sequence of Methods that follow ?

    onPause()
    onStop()
    onDestroy()

### What is the series of Methods that follow when a Dialog appears on top of Screen ?

    if covers it completely :
        onPause()
        onStop()
    if does not cover completely :
        onPause()

    But when the same activity comes to FOREGROUND :
        onRestart()
        onStart()
        onResume()

## Activity Basics

### What is an ACTIVITY in Android ?
     It is a focussed Screen in Apps, where the user can INTERACT WITH the elements of UI like the buttons, card, lists etc.
     It is basically an Entry Point of user interaction.

### What is the concept behind Activities in Android ?

    1. Entry Point for a application is not always fixed.
    2. an app may or may not be opened from the launcher activity always
    3. As an activity of an application can be opened from another app/system

### What are the 4 main Android Components ?

    1. Activity
    2. Services
    3. BroadCast Recv
    4. Content Provider

### Explain all the Activity lifecycle methods ?

    A. onCreate() :
        - Here write a logic which has to be executed only once for the Entire lifecycle of the application
        - example : Checking for App Updates, App Level DI etc...
        - @onLifeCycleEvent is c/a LifeCycle aware component.

        class MyOberver : LifeCycleObserver {
             @onLifeCycleEvent(LifeCycle.onCreate())
             fun onCreateEvent(){
                println("Do Something)
             }
        }

        - Register this oberserver class inside the main activity - lifecycle.addObservr(MyObserver())

    B. onStart() :
        - Activity is visible but not interActive

    D. onResume() :
        - Activity is visible and also interActive

    E. onPause() :
        - Activity is Partially visible.
        - exmaple : When An alert dialog appears
        - NOT advisable to save user data, make network calls or execute DB transactions as it does not give enough Time for these processes.

    F. onStop() :
        - Activity is completely hidden
        - release resources

    G. onDestry() :
        - Activity is finishing or being destryoed by system
        - Performs relatively cpu intensive shutdown operation here

    H. OnRestart() :
        - Activity being restarted after being stopped
        - Release all the resources

### What all does Component ACTIVITY do?
     1. Manages the UI components and responds to the User Interaction.
     2. Has its own LIFECYCLE METHODS
     3. Implements the following interfaces : Lifecycle, viewmodel, SavedStateHandle.
     4. Compose Ready (not tied to compose, only compose ready)

    Without it, we would have to manually implement the interfaces it implements like VM, savedStateHandle, lifecycle methods.

### What is the Activity Architecture we follow in Modern Android Development ?
    In Modern Android Dev, we follow a Single Activity Architecture. We have different Screens implemented as COMPOSABLES.

    Benefits :
    1. Easy Navigation Handling (don't have to manage each multiple activity back stacks)
    2. Shared ViewModels (Easy to Manage and share states as a ViewModel is tied to a ViewModelStoreOwner (usually an Activity))
    3. In multi-activity architecture, when you navigate to another activity, the old activity (and its ViewModel) gets destroyed unless you
        explicitly persist data somewhere else.
    4. No need to pass data between acivities using Intents.

### What is the Concept of Layout Inflation (like in XML) in Compose ?
    1. The set content {}  present un MainActivity takes a composable as the root composable
    2. It creates a ROOT COMPOSABLE inside the activity
    3. hooks the composable lifecycle to Activity Cycle
    4. Manages Recomposition when State Changes.

## BackStack and Tasks

### What happens when a user taps back button ?

    The following lifecycle methods are executed :
    1. onPause()
    2. onStop()
    3. onDestroy() and removed from the back stack

    Note : onSaveInstance() does not fire in this case, as the system doesn't expect the user to return to the same activity again.

### What happens to backstack when a new Activity is launched from another activity ?

    1. When A launched B, B is pushed to TOS and onStop() is called for Activtiy A
    2. Before onStop() is called, it retains its state.
    3. when pressed back, B is popped off the TOS, A then resumes & the state is restored.

### What happens when you press back on the MainActivity or the Launcher ?

    Borfore Android 11 :
        The system would finish the activity
    After :
        Sends it into Warn State instead of cold state where the users have to restart the App


    Note :
        1. using the AndroidX Activity Api, it auto defers to appropriate syatem behavior if there are no components overrididng the system back tap.
        2. Else if we use onBackPressed(), use super.onBackPressed(), as it moves the task to BG when appropriate.

### How do we define launch Modes ?

    There are 2 ways to define it.

    A. Using Manifest file -
        When we declare an act in manifest file, we can specify how the Act associates when it starts.

    B . Using Intent flag -
        After calling the startActivity() include flag in the Intent that declares how the new activity associates with the current task.

### Define the launch Modes in Manifest file ?

    A. Standard :
        - Default Mode
        - Act can be instantiated multi times.
        - each belong to different tasks that launch it.

    B. SingleTop :
        - if a instance already exists at TOS of the current task, the system Routes the intent to that instance through a call to its onNewIntent() Method.
        - example : Currentt Task - ABCD
            a new intent arrives for Activity D. So it would route the intent to the already existing D instance
            if a new instacne arrives for activity B, the stack would be A B C D B

    C. SingleTask :
        - only 1 existence of that act can exist in that TASK
        - if it already exists, it does not create a new one, instead it sends a newIntent to the existing instance.
        - All act above it are cleared (becomes the TOS)
        - if it doesn't exist, it creates a new one in a new task(becomes the root act of the new task)

    D. SingleInstance :
        - Similar to Singletask, except the system doesn't launch any other act into the task holding the instance
        - the act is the only member of the task
        - any act started by this act, opens in a different task
        -

    E. SingleInstance Per Task :
       - the act that runs only as root act of the task'
       - this activity can be started in multiple instance in different tasks if proper flag is set.
       - similar to singke task, it clears all the act above it
       - EXAMPLE - map : each route opens in different task


### Define launch Modes using the Intent Flags !!

    Note: The behaviors that you specify for your activity with the launchMode attribute can be overridden by flags included
            with the intent that start your activity.

    A. FLAG_ACTIVITY_NEW_TASK :
        - The system starts the activity in a new task.
        - If a task is already running for the activity being started, that task is brought to the foreground with its last state restored
        - the activity receives the new intent in onNewIntent().
        - This produces the same behavior as the "singleTask" launchMode value discussed in the preceding section

    B. FLAG_ACTIVITY_SINGLE_TOP :
        - If the activity being started is the current activity, at the top of the back stack, then the existing
            instance receives a call to onNewIntent() instead of creating a new instance of the activity.
        - This produces the same behavior as the "singleTop" launchMode value discussed in the preceding section

    C. FLAG_ACTIVITY_CLEAR_TOP :
        - If the activity being started is already running in the current task, then—instead of launching a new instance of that activity—the system destroys all the other activities on top of it. The intent is delivered to the resumed instance of the activity, now on top, through onNewIntent()
        - FLAG_ACTIVITY_CLEAR_TOP is most often used in conjunction with FLAG_ACTIVITY_NEW_TASK. When used together, these flags locate an existing
          activity in another task and put it in a position where it can respond to the intent.

# **Intent Related**

### What is an Intent ?
    Intents are MESSAGING objects, which are used to communicate with another App Components.

    USES -
    1. Start An Activity
    2. Run a service
    3. Deliver a broadcast

### What are the components of an intent ?

    A. Action
    B. Component Name
    C. Data
    D. Category
    E. Extras
    F. Flags

### What are the types of Intent ?
     A. Explicit -
        Used to communicate within the same app, example : To Move between different activties or services

        // Executed in an Activity, so 'this' is the Context
        // The fileUrl is a string URL, such as "http://www.example.com/image.png"
        val downloadIntent = Intent(this, DownloadService::class.java).apply {
            data = Uri.parse(fileUrl)
        }
        startService(downloadIntent)


     B. Implicit -
        - Declares a general action to perform & Android Decides which Component can handle it (shows options when you click on pdf, diff apps are shown)
        - When you use an implicit intent, the Android system finds the appropriate component to start by comparing the contents of the
            intent to the intent filters declared in the manifest file of other apps on the device


        // Create the text message with a string.
        val sendIntent = Intent().apply {
            action = Intent.ACTION_SEND
            putExtra(Intent.EXTRA_TEXT, textMessage)
            type = "text/plain"
        }

        // Try to invoke the intent.
        try {
            startActivity(sendIntent)
        } catch (e: ActivityNotFoundException) {
            // Define what your app should do if no activity can handle the intent.
        }

     Other Types -
     1. BroadCast Intent : Used to deliver system wide app announcements
     2. Pending Intent : For future Work, like alarms, scheduled notification

### What are some actions to start some activity ?

    A. ACTION_VIEW
        - Use this action in an intent with startActivity() when you have some information that an activity can show to the user, such as a photo to
            view in a gallery app, or an address to view in a map app.

    B. ACTION_SEND
        - Also known as the share intent, you should use this in an intent with startActivity() when you have some data that the user
            can share through another app, such as an email app or social sharing app.

    and many more ...



### What is Component Name ?

    The name of the component to start.

     - This is optional, but it's the critical piece of information that makes an intent explicit,
     - meaning that the intent should be delivered only to the app component defined by the component name.
     - Without a component name, the intent is implicit and the system decides which component should receive the intent
        based on the other intent information (such as the action, data, and category—described below).
     - If you need to start a specific component in your app, you should specify the component name.

### Data in Intents !

    - When creating an intent, it's often important to specify the type of data (its MIME type) in addition to its URI.
    - example, an activity that's able to display images probably won't be able to play an audio file, even though the URI formats could be similar.
    - Specifying the MIME type of your data helps the Android system find the best component to receive your intent.
    - To set only the data URI, call setData(). To set only the MIME type, call setType(). If necessary, you can set both explicitly with setDataAndType().

### what are the categories in Intent ?

    - A string containing additional information about the kind of component that should handle the intent.
    - Any number of category descriptions can be placed in an intent, but most intents do not require a category. Here are some common categories:

    CATEGORY_BROWSABLE
        The target activity allows itself to be started by a web browser to display data referenced by a link, such as an image or an e-mail message.
    CATEGORY_LAUNCHER
        The activity is the initial activity of a task and is listed in the system's application launcher.

    You can specify a category with addCategory()

# **Services Related**

### What are services ?
    Taskes that run in the Background to perform Long Running Operations without the UI. Useful for taks which should continue even if the app is closed
    or user does not interact with App.

    Examples : Background Music Play, Downloading something, Syncing contacts with the server

### What are the types of services ?
    A. ForeGround :
        Runs in foreground with persistent notification
        ex : Downlaod Manager Showing Progess

    B. BackGround :
        Runs Until stopped
        ex : Music Playback in background.

    # **BroadCast related**

### What is a broadcast ?
    It is a system wide message that any apps or components can send or receive about an event or trigger.

### What is a broadcast Receiver?
    Android Component that allows an app to lsiten to system wide events without needing to run in background.
    BoradCast recv listens for any such messages runs a code when it comes.

    example : Battery Low, Connectivty Change, On boot Complete.


### What are the types of BroadCast Receivers ?
    A. Static :
        Declared Inside Manifest. ex : On boot completed.
    B. Dynamic :
        Written inside code. ex : Connectivity Change.

# **Content Provider Related**

### What is Content Provider ?
    Android Component that manages & shares structured data B/W apps. Acts like a middleMan.

    Uses A URI(Uniform Resource Id.)
    Access cam be restricted using Permissions

    Example :
        The contact App provides Contact data through Contacs Contract Provider. Whatsapp can query it to get the comtacts ONLY IF
        it has the required PERMISSIONS.

# **Navigation Based**

## Nested Graph

### Explain how the nested navigation works ?


    @Composable
    fun RootNavGraph(
        authViewModel: AuthViewModel,
        userViewModel: UserViewModel,
        connectViewModel: ConnectViewModel,
        connectChatViewModel: ConnectChatViewModel,
        messageViewModel: MessageViewModel
    ) {

    val rootNavController = rememberNavController()

    NavHost(navController = rootNavController, route = Graphs.ROOTGRAPH,
        startDestination = "something"
    ){
        authGraph(rootNavController, authViewModel, userViewModel) // ----------- this is a NESTED graph
        onBoardingGraph(rootNavController,authViewModel,userViewModel)

        // ------- this is a seperate NavHost

        composable(Graphs.MAINNAVGRAPH) { MainScreen(rootNavController, authViewModel, userViewModel,connectViewModel, connectChatViewModel, messageViewModel) }

        composable(
            route = ConnectChatDetailScreen.DetailScreen.route,
            arguments = listOf(
                navArgument("chatId") { type = NavType.StringType }
            )
        ) { backStackEntry ->
            val chatId = backStackEntry.arguments?.getString("chatId") ?: ""
            ConnectChatScreen(chatId, rootNavController,connectViewModel,connectChatViewModel, userViewModel, messageViewModel)
        }
    }
    }

    // NESTED graph
    fun NavGraphBuilder.authGraph(
        rootNavController: NavHostController,
        authViewModel: AuthViewModel,
        userViewModel: UserViewModel
    )
    {


        navigation(route = Graphs.AUTHGRAPH, startDestination = AuthScreens.PhoneVerification.route){
            composable(AuthScreens.PhoneVerification.route){
                PhoneVerificationScreen(rootNavController, authViewModel, userViewModel)
            }
        }
    }

    // seperate NAVHOST
    @Composable
    fun MainNavGraph(
        mainNavController: NavHostController,
        rootNavController: NavHostController,
        authViewModel: AuthViewModel,
        userViewModel: UserViewModel,
        connectViewModel: ConnectViewModel,
        connectChatViewModel: ConnectChatViewModel,
        messageViewModel: MessageViewModel
    ) {
        NavHost(
             navController = mainNavController,
            route = Graphs.MAINNAVGRAPH,
            startDestination = MainNavScreens.ChatScreen.route
        ) {
            composable(MainNavScreens.ChatScreen.route) { HomeScreen(mainNavController, rootNavController) }
            composable(MainNavScreens.ConnectScreen.route) { ConnectScreen(mainNavController, rootNavController,connectViewModel,userViewModel, connectChatViewModel) }
            composable(MainNavScreens.ProfileScreen.route) { ProfileScreen(mainNavController, rootNavController, authViewModel, userViewModel) }
            composable(MainNavScreens.HistoryScreen.route) { HistoryScreen(mainNavController, rootNavController, connectViewModel,userViewModel, connectChatViewModel) }
            composable(MainNavScreens.NotificationScreen.route) { InboxScreen(mainNavController, rootNavController) }
        }
    }



## DeepLink Based

### What is a deep link in android ?

    - In Android, a deep link is a link that takes you directly to a specific destination within an app.
    - The Navigation component lets you create two different types of deep links: explicit and implicit.

     - compose example : composable(
          "profile/{userId}",
          deepLinks = listOf(navDeepLink { uriPattern = "https://www.example.com/profile/{userId}" })
        ) { backStackEntry ->
          val id = backStackEntry.arguments?.getString("userId")
          ProfileScreen(userId = id)
        }

    - <activity android:name=".ProfileActivity">
      <intent-filter>
        <action android:name="android.intent.action.VIEW" />
        <category android:name="android.intent.category.DEFAULT" />
        <category android:name="android.intent.category.BROWSABLE" />
        <data android:scheme="https"
              android:host="www.example.com"
              android:pathPrefix="/profile" />
      </intent-filter>
    </activity>

    1. Explicit Deep Links

        These are direct links to a specific destination in your app’s navigation graph.

        You define them in the navigation graph XML (or in code) and they always know exactly which screen to open.

        Example:

        You add a deep link to ChatScreen in your navigation graph.

        If an intent with app://myapp/chat/123 is received, Navigation will directly navigate to ChatScreen with the correct arguments.

        ✅ Always reliable, used when you know the exact destination.

    2. Implicit Deep Links

        These are intent filters you declare in your app’s manifest (AndroidManifest.xml).

        They allow your app to respond to external URLs or app links.

        Example:

        You add an intent filter for https://www.myapp.com/profile/*.

        When the user clicks such a link from email, browser, or another app, Android checks which apps can handle it. If your app is installed, the system can launch it.

        ✅ Good for external entry points into your app.



### how do you handle android App Links?

    Deep links :
        - Deep links are URIs of any scheme that take users directly to a specific part of your app. To create deep links,
        add intent filters to drive users to the right activity in your app

    Web links :
        - Web links are deep links that use the HTTP and HTTPS schemes.
        they'll see a disambiguation dialog

    Android App Links :
        - Android App Links, available on Android 6.0 (API level 23) and higher, are web links that use
            the HTTP and HTTPS schemes and contain the autoVerify attribute.
        - This attribute allows your app to designate itself as the default handler of a given type of link

## Type Safe Navigation

### What is type safe navigation and its benefits ?

    // Define a home route that doesn't take any arguments
    @Serializable
    object Home

    // Define a profile route that takes an ID
    @Serializable
    data class Profile(val id: String)

    NavHost(navController, startDestination = Home) {
         composable<Home> {
             HomeScreen(onNavigateToProfile = { id ->
                 navController.navigate(Profile(id))
             })
         }
         composable<Profile> { backStackEntry ->
             val profile: Profile = backStackEntry.toRoute()
             ProfileScreen(profile.id)
         }
    }

    navController.navigate(Profile(id = 123))


### Encapsulate navigation events

    - Encapsulate your navigation code for a related set of screens by placing it in a separate file
    - Expose destinations by creating extension functions on NavGraphBuilder
    - Expose navigation events by creating extension functions on NavController

    // ContactsNavigation.kt

    fun NavController.navigateToContactDetails(id: String) {
      navigate(route = ContactDetails(id = id))
    }

    // MyApp.kt

    @Composable
    fun MyApp() {
      ...
      NavHost(navController, startDestination = Contacts) {
         contactsDestination(onNavigateToContactDetails = { contactId ->
            navController.navigateToContactDetails(id = contactId)
         })
         contactDetailsDestination()
      }
    }



### Never Pass navController to your composables

    Warning: Don't pass your NavController to your composables. Expose an event instead.